##In Colab, be sure to select 'GPU' under 'Edit' -> 'Notebook Settings' -> 'Hardware accelerator'.

First, necessary packages are loaded and repository is cloned

In [ ]:
import torch
import torchvision
import sys
import os
os.chdir('/content')
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

!{sys.executable} -m pip install opencv-python ultralytics==8.0.120

!git clone https://gitlab.au.dk/disorder/np-sam.git

os.chdir('/content/np-sam')

!rm LICENSE.txt NPSAM.ipynb README.md requirements.txt

!wget -N https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

from Utils import *

Upload the image(s) to to the content folder and set a pathname called `filepath` to a file or a folder of the filetype .png, .jpg, .jpeg, .tif, .tiff (either an absolute or relative pathname)

In [ ]:
filepath = '/content/np-sam/Sample_images/PdCu_HAADF.png'

In [ ]:
filepath = '/content/np-sam/Sample_images'

Call the `SAM` function from `Utils.py` to get masks from the image. The function takes arguments:<br>
    `device` (str): Can be either `cuda` or `cpu`. <br>
    `PPS` (int): Sets the sampling gridsize for the image (`default=64`). <br>
    `use_FastSAM` (bool): If True the model will opt to use [FastSAM](https://github.com/CASIA-IVA-Lab/FastSAM) which is a lot faster but can struggle with more complicated samples and small NPs (`default=False`). <br>
    `pre_filter` (bool): Removes faulty masks for a better and faster experience. However, for non-spherical NPs it might cause problems (`default=True`) <br>
    `min_masks_region_area` (int): Sets the minimum mask area in pixels. Makes SAM faster but will miss NPs smaller than threshold (`default=35`) <br>
    `**kwargs`: More key word arguments can be applied see [documentation](https://github.com/facebookresearch/segment-anything/blob/main/segment_anything/automatic_mask_generator.py#L35) of the model. <br>
When executing `SAM` a folder `_files` is created either besides the single file or inside the folder of the multiple files.

In [3]:
SAM(filepath, device='cuda', PPS=64, use_FastSAM=True)

***
If `SAM` has already been executed once it is not necessary to segment the image again, as the masks have previously been saved, that it unless you want to redo it with other parameters. I Google Colab the runtime will expire quite fast if the user is inactive and therefore it is a good idea to save the files made by `SAM` locally then they can be further processed in Jupyter lab or the user can continue in Google Colab.

The `nb_unique` function from `Utils.py` is used to find every unique binary vector, or in other words every unique combination of masks, of the sampled pixels. <br>
`stepsize` (int): Sets the step size such that the function `nb_unique` will traverse the array of masks faster. With increasing step size the faster `nb_unique` will run. However, be cautious to not increase the step size too much for images with small NPs.

In [ ]:
nb_unique(filepath)

***
The `labels_func` from `Utils.py` gives all pixels sharing the same unique binary vector of masks a unique label (an integer) to be easily identified later. In this function another function from `scikit-image` called `remove_small_objects`from the file `morphology`is envoked. It removes all not-connected grains of pixels smaller than a given area which in our case is set to 16 pixels.

In [ ]:
labels_func(filepath)

In [ ]:
label_plot(filepath)

The function `properties` will create a pandas dataframe that contains properties from the `scikit-image`library `regionprops` for more [infomation](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops).
The function takes the argument: <br>
`scaling` (bool): which if True prompts the user to apply a scaling i pixels pr nm. Can also take a fraction: Therefore, if the image has a $50$ nm scalebar measuring $400$ pixels a scale can be given as $400/50$.
For multiple files a specific scaling can be given for eack file or the same for all.
The initial properties are saved as a csv file called `_raw_properties` such external editing is possible

In [ ]:
properties(filepath, scaling=True)

The interactive GUI is not available in Google Colab. Therefore, filter conditions can be applied manually. Be aware to use the same syntax as shown below including spaces, ' and </>.

In [ ]:
conditions = """[
(df['solidity'] > 0.2),
(df['area'] > 1),
(df['eccentricity'] < 0.9),
(df['overlap'] < 30),
(df['intensity_mean'] < 200),
(df['euler_number'] < 2)
]"""

In [ ]:
define_manual(filepath)

If `image_number=0` the conditions are applied to all images otherwise specify which image you want to filter.

In [ ]:
manual_filter(filepath, conditions, image_number=1)

Conclusively, an overview can be made with said function. It plots all images with a histogram over the area, and which parameters was use to filter the labels. If a folder has been used the united area histogram is plotted. This is then compiled to a pdf.

In [ ]:
overview(filepath)